In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader,Dataset, WeightedRandomSampler
import os
from PIL import Image
import pandas as pd
from torch.optim import lr_scheduler
from torch.utils.data import random_split
from collections import Counter


In [18]:

def make_weights_for_balanced_classes(images, nclasses):
    count = [0] * nclasses
    for item in images:
        count[item[1]] += 1
    weight_per_class = [0.] * nclasses
    N = float(sum(count))
    for i in range(nclasses):
        weight_per_class[i] = N/float(count[i])
    weight = [0] * len(images)
    for idx, val in enumerate(images):
        weight[idx] = weight_per_class[val[1]]
    return weight

In [19]:
data_dir="/content/drive/MyDrive/"

In [20]:
image_folder_train = os.path.join(data_dir,"md_hw2/dataset/training")
image_folder_test = os.path.join(data_dir,"md_hw2/dataset/test")
image_paths_train =[os.path.join(image_folder_train,filename) for filename in os.listdir(image_folder_train) if filename.endswith(".jpg")]
image_paths_test= [os.path.join(image_folder_test, filename) for filename in os.listdir(image_folder_test) if filename.endswith(".jpg")]

labels_file_train = os.path.join(data_dir, "md_hw2/dataset/training_labels.txt")
labels_file_test= os.path.join(data_dir, "md_hw2/dataset/test_labels.txt")
labels_df_train = pd.read_csv(labels_file_train, delimiter=" ", header=None, names=["Label"])
labels_df_test = pd.read_csv(labels_file_test, delimiter=" ", header=None, names=["Label"])
labels_df_train["Label"] -= 1
labels_df_test["Label"] -= 1

train_indices = range(0, len(image_paths_train))
test_indices = range(0, len(image_paths_test))



In [21]:
train_image_paths = [image_paths_train[i] for i in train_indices]
train_labels = [labels_df_train.loc[i, "Label"] for i in train_indices]

test_image_paths = [image_paths_test[i] for i in test_indices]
test_labels = [labels_df_test.loc[i, "Label"] for i in test_indices]


In [22]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label



In [33]:

transform = transforms.Compose([
    transforms.Resize(224),
 #   transforms.RandomResizedCrop(224),
#    transforms.RandomHorizontalFlip(),
 #   transforms.RandomRotation(30),
    transforms.ToTensor(),
   # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    transforms.Normalize([0.7108, 0.6784, 0.7242], [0.1603, 0.1857, 0.1047])
])

transform_test=transforms.Compose([
   transforms.Resize(224),
  # transforms.CenterCrop((224,224)),
   transforms.ToTensor(),
 #  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
   transforms.Normalize([0.7117, 0.6809, 0.7257], [0.1587, 0.1835, 0.1033])

])



In [41]:
train_dataset = CustomDataset(train_image_paths, train_labels, transform=transform)
test_dataset = CustomDataset(test_image_paths, test_labels, transform=transform_test)
#weights = make_weights_for_balanced_classes(train_dataset,3)
#sampler = WeightedRandomSampler(weights, len(train_dataset), replacement=True)

train_dataset, val_dataset = random_split(train_dataset, [len(train_dataset) - 38, 38])

weights = make_weights_for_balanced_classes(train_dataset,3)
sampler = WeightedRandomSampler(weights, len(train_dataset), replacement=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, sampler=sampler)
#train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False)
val_loader=torch.utils.data.DataLoader(val_dataset,batch_size=4,shuffle=True)



In [42]:
model=torchvision.models.alexnet(pretrained=True)

for param in model.features.parameters():
    param.requires_grad=False



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [43]:

num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 3)


In [13]:
print(model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [44]:

criterion=torch.nn.CrossEntropyLoss()
#criterion=torch.nn.functional.cross_entropy
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [45]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    class_correct_train = [0] * 3
    class_total_train = [0] * 3
#    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
#        class_weights=class_weights.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        for label, prediction in zip(labels, predicted):
            if label == prediction:
                class_correct_train[label] += 1
            class_total_train[label] += 1

        if (i+1) % 100 == 0:
           # print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
           print('Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))
    scheduler.step()
    epoch_loss = running_loss / len(train_dataset)
    train_acc = correct / total
#    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Accuracy: {:.4f}'.format(epoch+1, num_epochs, epoch_loss, train_acc))

    model.eval()
    correct = 0
    total = 0
    class_correct_val = [0] * 3
    class_total_val = [0] * 3
    with torch.no_grad():
        for i, (inputs,labels) in enumerate(val_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for label, prediction in zip(labels, predicted):
                if label == prediction:
                    class_correct_val[label] += 1
                class_total_val[label] += 1

    val_acc = correct / total
    print('Epoch [{}/{}], Validation Accuracy: {:.4f}'.format(epoch+1, num_epochs, val_acc))
    scheduler.step()
print('Training finished.')

correct=0
total=0
true_positives = 0
false_positives = 0
false_negatives = 0
class_correct = [0] * 3
class_total = [0] * 3
model.eval()
with torch.no_grad():
    for i, (inputs,labels) in enumerate(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Get predictions
        _, predicted = torch.max(outputs, 1)


        # Update counts
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        for label, prediction in zip(labels, predicted):
            if label == prediction:
                class_correct[label] += 1
            class_total[label] += 1

        true_positives += ((predicted == 1) & (labels == 1)).sum().item()
        false_positives += ((predicted == 1) & (labels == 0)).sum().item()
        false_negatives += ((predicted == 0) & (labels == 1)).sum().item()
accuracy = correct / total
print(correct)
print(total)
print('Test Accuracy: {:.4f}%'.format(accuracy))

precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

# Calculate Recall
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Calculate F1-score
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print("Correct:", correct)
print("Total:", total)
print('Test Accuracy: {:.4f}%'.format(accuracy))
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1_score)

print("Class Accuracies - Training:")
for i in range(3):
        class_accuracy_train = class_correct_train[i] / class_total_train[i] if class_total_train[i] > 0 else 0
        print('Class {} Accuracy: {:.4f}%'.format(i, class_accuracy_train))

print("Class Accuracies - Validation:")
for i in range(3):
        class_accuracy_val = class_correct_val[i] / class_total_val[i] if class_total_val[i] > 0 else 0
        print('Class {} Accuracy: {:.4f}%'.format(i, class_accuracy_val))

print("Class Accuracies - Test:")
for i in range(3):
    class_accuracy = class_correct[i] / class_total[i] if class_total[i] > 0 else 0
    print('Class {} Accuracy: {:.4f}%'.format(i, class_accuracy))


Epoch [1/20], Train Loss: 0.4632, Train Accuracy: 0.8716
Epoch [1/20], Validation Accuracy: 0.9737
Epoch [2/20], Train Loss: 0.1903, Train Accuracy: 0.9459
Epoch [2/20], Validation Accuracy: 0.9737
Epoch [3/20], Train Loss: 0.2461, Train Accuracy: 0.9257
Epoch [3/20], Validation Accuracy: 0.6842
Epoch [4/20], Train Loss: 0.3482, Train Accuracy: 0.9122
Epoch [4/20], Validation Accuracy: 0.8421
Epoch [5/20], Train Loss: 0.0993, Train Accuracy: 0.9527
Epoch [5/20], Validation Accuracy: 0.9474
Epoch [6/20], Train Loss: 0.0959, Train Accuracy: 0.9730
Epoch [6/20], Validation Accuracy: 0.9474
Epoch [7/20], Train Loss: 0.0779, Train Accuracy: 0.9865
Epoch [7/20], Validation Accuracy: 0.9474
Epoch [8/20], Train Loss: 0.0178, Train Accuracy: 1.0000
Epoch [8/20], Validation Accuracy: 0.9474
Epoch [9/20], Train Loss: 0.0496, Train Accuracy: 0.9730
Epoch [9/20], Validation Accuracy: 0.9474
Epoch [10/20], Train Loss: 0.0462, Train Accuracy: 0.9865
Epoch [10/20], Validation Accuracy: 0.9474
Epoch [1